In [7]:
def read_table(spark, database_name, table_name):

    df = spark.table(f"{database_name}.{table_name}")
    
    return df

def create_hive_external_table(spark, df, database_name, table_name, partition_column_name):
    
    spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
    df.write.partitionBy(partition_column_name).mode("overwrite").option('path',f'hdfs://namenode:8020/user/admin/hive/{database_name}/{table_name}').saveAsTable(f'{database_name}.{table_name}')
    
def calculating_volatility(spark):
    
    df = read_table(spark, "bronze", "cryptodata_raw")
    
    windowSpec  = Window.partitionBy("currency").orderBy("currency")


    df = df.withColumn("priceUsd_lagged",lag("priceUsd",1).over(windowSpec))
    df = df.withColumn("volatility", (F.col("priceUsd") - F.col("priceUsd_lagged")) / F.col("priceUsd_lagged"))
    df = df.withColumn("volatility_percentage", (F.col("volatility") * 100))
    df = df.select(F.col("date"), F.col("time"), F.col("priceusd").cast("double"), F.col("circulatingsupply").cast("double"), F.col("priceusd_lagged").cast("double"), F.col("volatility"), F.col("volatility_percentage"), F.col("currency"))
    
    create_table_in_hdfs(spark , df, "silver", "silver_data", "currency")